In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [5]:
#upload the dataset by downloading both datasets from canvas and upload it on colab
data_df = pd.read_csv("dataset_exercise_5_clustering_highway_traffic.csv",sep=";")
data_df

,PORTAL,Date,time_from,time_to,Interval_5,SPEED_MS_AVG,flow
0,"E4S 56,780",20210101,00:00:00,00:05:00,0,18.56,39
1,"E4S 56,780",20210101,00:05:00,00:10:00,1,20.39,18
2,"E4S 56,780",20210101,00:10:00,00:15:00,2,19.27,26
3,"E4S 56,780",20210101,00:15:00,00:20:00,3,19.52,52
4,"E4S 56,780",20210101,00:20:00,00:25:00,4,20.52,52
...,...,...,...,...,...,...,...
104838,"E4S 56,780",20211231,23:35:00,23:40:00,283,19.58,115
104839,"E4S 56,780",20211231,23:40:00,23:45:00,284,19.47,87
104840,"E4S 56,780",20211231,23:45:00,23:50:00,285,19.77,130
104841,"E4S 56,780",20211231,23:50:00,23:55:00,286,18.79,129


In [7]:
data_2_df = pd.read_csv("Dataset_2.csv",sep=",")
data_2_df

,Price,Safety,Exterior_Looks,Space_comfort,Technology,After_Sales_Service,Resale_Value,Fuel_Type,Fuel_Efficiency,Color,Maintenance,Test_drive,Product_reviews,Testimonials
0,4,4,5,4,3,4,5,4,4,2,4,2,4,3
1,3,5,3,3,4,4,3,4,3,4,3,2,2,2
2,4,4,3,4,5,5,5,4,5,4,5,4,4,3
3,4,4,4,3,3,4,5,5,4,4,4,2,5,3
4,5,5,4,4,5,4,5,3,4,5,5,5,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,4,4,3,4,5,4,4,4,4,4,5,4,5,4
86,4,3,4,4,5,4,3,4,4,5,4,4,5,4
87,4,3,4,4,4,5,3,4,4,3,3,2,4,4
88,4,4,3,4,4,5,3,4,4,4,4,4,4,5


In [11]:
# Sort the DataFrame 'data_df' by columns "Date" and "Interval_5"
data_df.sort_values(["Date", "Interval_5"])
# Extract unique dates from the sorted DataFrame
days = np.unique(data_df[['Date']].values.ravel())
# Calculate the total number of unique days
ndays = len(days)
# Group the DataFrame 'data_df' by the "Date" column
day_subsets_df = data_df.groupby(["Date"])
# Define the total number of 5-minute intervals in a day
nintvals = 288

# Create a matrix 'vectorized_day_dataset' filled with NaN values
vectorized_day_dataset = np.zeros((ndays, nintvals))
vectorized_day_dataset.fill(np.nan)
# Loop through each unique day
for i in range(0, ndays):
    # Get the DataFrame corresponding to the current day
    df_t = day_subsets_df.get_group(days[i])
    # Loop through each row in the current day's DataFrame
    for j in range(len(df_t)):
        # Get the current day's DataFrame
        df_t = day_subsets_df.get_group(days[i])
        # Extract the "Interval_5" and "flow" values and populate 'vectorized_day_dataset'
        vectorized_day_dataset[i, df_t.iloc[j]["Interval_5"]] = df_t.iloc[j]["flow"]
# Print the resulting 'vectorized_day_dataset' and the size of vector


# print the number of days with missing value
nans_per_day = np.sum(np.isnan(vectorized_day_dataset),1)
print('number of days with missing value:',np.size(np.where(nans_per_day > 0),1))

# Drop the days with missing valus
vectorized_day_dataset_no_nans = vectorized_day_dataset[np.where(nans_per_day == 0)[0],:]
# days_not_nans = days[np.where(nans_per_day == 0)[0]]
print(vectorized_day_dataset_no_nans.shape)

number of days with missing value: 28
(337, 288)


In [12]:
data = pd.DataFrame(vectorized_day_dataset_no_nans)
data.head()  # Display the first few rows
data.describe()

,0,1,2,3,4,5,6,7,8,9,...,278,279,280,281,282,283,284,285,286,287
count,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,...,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000
mean,33.314540,32.985163,32.566766,31.771513,31.545994,29.896142,28.798220,27.673591,26.836795,26.332344,...,44.919881,45.931751,44.670623,42.902077,40.703264,39.501484,37.543027,36.169139,35.952522,33.753709
std,15.895554,15.590837,15.211089,15.052824,14.602423,14.448942,14.522794,13.478609,13.888804,13.761443,...,19.550670,19.604682,18.469283,18.352514,18.369741,18.716577,17.610673,17.948285,18.085877,16.511590
min,5.000000,8.000000,6.000000,3.000000,6.000000,7.000000,6.000000,7.000000,3.000000,8.000000,...,4.000000,4.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000
25%,22.000000,22.000000,22.000000,20.000000,21.000000,19.000000,18.000000,19.000000,17.000000,17.000000,...,31.000000,31.000000,32.000000,30.000000,29.000000,26.000000,24.000000,23.000000,23.000000,22.000000
50%,29.000000,29.000000,29.000000,29.000000,28.000000,26.000000,26.000000,25.000000,23.000000,23.000000,...,43.000000,44.000000,42.000000,41.000000,38.000000,36.000000,34.000000,32.000000,33.000000,31.000000
75%,41.000000,42.000000,40.000000,40.000000,39.000000,38.000000,36.000000,34.000000,32.000000,32.000000,...,58.000000,59.000000,55.000000,54.000000,50.000000,48.000000,46.000000,45.000000,46.000000,42.000000
max,101.000000,83.000000,92.000000,81.000000,83.000000,78.000000,73.000000,109.000000,91.000000,121.000000,...,107.000000,101.000000,103.000000,104.000000,103.000000,115.000000,101.000000,130.000000,129.000000,117.000000
